# Building a conversational chatbot using Rasa

## Import statements

In [89]:
import logging, json, warnings
warnings.filterwarnings('ignore')
logging.basicConfig(level='INFO')

import rasa.nlu
import rasa.core
import spacy

## NLU training data

In [129]:
nlu_md = '''
## intent:greet
- Hi
- Hey
- Hi bot
- Hey bot
- Hello
- Good morning
- hi again
- hi folks
- hi Mister
- hi pal!
- hi there
- greetings
- hello everybody
- hello is anybody there
- hello robot
- hallo
- heeey
- hi hi
- hey
- hey hey
- hello there
- hi
- hello
- yo
- hola
- hi?
- hey bot!
- hello friend

## intent:request_flight
- im looking for a flight
- i want to go on a trip
- i want to book a flight
- i want to book a trip
- need a ticket 
- help me find a flight
- get me out of here
- i want to travel
- i want to fly somewhere
- can i get a flight
- find me a flight
- get me a flight
- i want to go on a flight 
- i need a flight 
- i want a flight from [new york] (origin) to [toronto] (destination)
- get me a flight from [seattle] (origin) to [denver] (destination)

## intent:affirm
- correct
- ye
- uh yes
- let's do it
- yeah
- uh yes
- um yes
- that's correct
- yes yes
- right
- yea
- yes
- yes right
- yes and i dont care
- right on
- i love that
- perfect
- cool
- dope
- great

## intent:deny
- no
- no new selection
- no thanks
- no thank you
- uh no
- breath no
- do you have something else
- no this does not work for me
- not what i want

## intent:inform
- i'm coming from [toronto] (origin)
- im coming from [new york] (origin)
- starting from [vancouver] (origin)
- start from [los angeles] (origin)
- leave from [chicago] (origin)
- leaving from [dallas] (origin)
- departing from [denver] (origin)
- depart from [seattle] (origin)
- from [miami] (origin)
- i'm going to [florida] (destination)
- im going to [tokyo] (destination)
- i want to go to [new york] (destination)
- i want to get to [denver] (destination)
- to [tokyo] (destination)
- lets go to [miami] (destination)
- let's go to [seattle] (destination)
- i want to go to [new york] (destination) from [toronto] (origin)
- get me from [miami] (origin) to [dallas] (destination)
- im planning on leaving from [august 14] (depart_date) to [september 14] (return_date)
- trip is planned from [august 14] (depart_date) to [september 14] (return_date)
- i want to go from [jan 14] (depart_date) to [apr 14] (return_date)
- leave on [may 14] (depart_date) and get back [june 14] (return_date)
- leave on [may 4] (depart_date)
- i want to leave on [january 12] (depart_date)
- i planned to leave on [march 8]
- come back [may 18] (return_date)
- return on [february 9] (return_date)
- planned on being back [april 18] (return_date)
- my budget is [$2000] (budget)
- no more than [1000] (budget)
- max [$400] (budget)
- [100 dollars] (budget)
- 100 - [200] (budget)

## intent:thankyou
- um thank you good bye
- okay cool uh good bye thank you
- okay thank you good bye
- you rock
- and thats all thank you and good bye
- thank you and good bye
- sorry about my mistakes thank you good bye
- okay thank you goodbye
- uh thank you good bye
- thank you goodbye
- okay thank you
- thanks goodbye
- ah thank you goodbye
- thank you noise
- thank you good bye
- thanks
- noise thank you goodbye
- uh okay thank you good bye
- thank you bye
- um okay thank you good bye

## intent:chitchat
- i want to know the company which designed you
- i want to know the company which generated you
- i want to know the company which invented you
- i want to know who invented you
- May I ask who invented you?
- please tell me the company who created you
- please tell me who created you
- tell me more about your creators
- tell me more about your founders
- Ahoy matey how are you?
- are you alright
- are you having a good day
- Are you ok?
- are you okay
- Do you feel good?
- how are things going
- how are things with you?
- How are things?
- how are you
- how are you doing
- how are you doing this morning
- how are you feeling
- how are you today
- How are you?

## intent:stop
- ok then you cant help me
- that was shit, you're not helping
- you can't help me
- you can't help me with what i need
- i guess you can't help me then
- ok i guess you can't help me
- that's not what i want
- ok, but that doesnt help me
- this is leading to nothing
- this conversation is not really helpful
- you cannot help me with what I want
- I think you cant help me
- hm i don't think you can do what i want
- stop
- stop go back
- and that's it?
- nothing else?

## intent:bot_challenge
- are you a bot?
- are you a human?
- am I talking to a bot?
- am I talking to a human?
'''

%store nlu_md > rasa_bot/data/nlu.md

Writing 'nlu_md' (str) to file 'rasa_bot/data/nlu.md'.


## NLU model configuration

In [130]:
config = '''
language: en

pipeline:
  - name: "SpacyNLP"
  - name: "SpacyTokenizer"
  - name: "SpacyFeaturizer"
  - name: "RegexFeaturizer"
  - name: "CRFEntityExtractor"
  - name: "EntitySynonymMapper"
  - name: "SklearnIntentClassifier"

policies:
  - name: FallbackPolicy
  - name: MemoizationPolicy
  - name: FormPolicy
  - name: KerasPolicy
  - name: MappingPolicy
'''

%store config > rasa_bot/config.yml

Writing 'config' (str) to file 'rasa_bot/config.yml'.


## Train NLU model

In [131]:
from rasa.nlu.training_data import load_data
from rasa.nlu.config import RasaNLUModelConfig
from rasa.nlu.model import Trainer
from rasa.nlu import config

# loading the nlu training samples
training_data = load_data('rasa_bot/data/nlu.md')

# trainer to educate our pipeline
trainer = Trainer(config.load('rasa_bot/config.yml'))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("rasa_bot/models/nlu", fixed_model_name="current")

INFO:rasa.nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa.nlu.model:Starting to train component SpacyNLP
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component SpacyTokenizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component SpacyFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component RegexFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component CRFEntityExtractor
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component EntitySynonymMapper
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component SklearnIntentClassifier
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.3s finished
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Successfully saved model into '/Users/gthorani/unit_6/travel_chatbot/rasa_bot/models/nlu/current'


## Let's evaluate the NLU model with a sample text

In [136]:
def pprint(o):   
    print(json.dumps(o, indent=2))
    
pprint(interpreter.parse('i want to book a flight'))

{
  "intent": {
    "name": "request_flight",
    "confidence": 0.5948283489640181
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "request_flight",
      "confidence": 0.5948283489640181
    },
    {
      "name": "stop",
      "confidence": 0.09975164720756306
    },
    {
      "name": "affirm",
      "confidence": 0.08410848197427101
    },
    {
      "name": "chitchat",
      "confidence": 0.05761135249773464
    },
    {
      "name": "deny",
      "confidence": 0.04644334670203227
    },
    {
      "name": "greet",
      "confidence": 0.04191204060700699
    },
    {
      "name": "inform",
      "confidence": 0.03409126234327976
    },
    {
      "name": "bot_challenge",
      "confidence": 0.03248589640795895
    },
    {
      "name": "thankyou",
      "confidence": 0.008767623296135124
    }
  ],
  "text": "i want to book a flight"
}


- We observe here that the user's intent is to request a flight

## Evaluating NLU model on test data

(Definitely want to use unseen data, but will be using the data we have)

In [108]:
from rasa.nlu.test import run_evaluation

#run_evaluation('rasa_bot/data/nlu.md', model_directory)

## Using Rasa core to teach the bot to respond

In [115]:
stories_md = '''
## happy path
* greet
    - utter_greet
* request_flight
    - flight_form
    - form{"name": "flight_form"}
    - form{"name": null}
    - utter_slots_values
* thankyou
    - utter_noworries
    
## chitchat stop but continue path
* request_flight
    - flight_form
    - form{"name": "flight_form"}
* chitchat
    - utter_chitchat
    - flight_form
* stop
    - utter_ask_continue
* affirm
    - flight_form
    - form{"name": null}
    - utter_slots_values
* thankyou
    - utter_noworries

## bot challenge
* bot_challenge
  - utter_iamabot
'''
%store stories_md > rasa_bot/data/stories.md

Writing 'stories_md' (str) to file 'rasa_bot/data/stories.md'.


## Define domain

In [113]:
domain_yml = '''
intents:
  - request_flight:
      use_entities: []
  - chitchat:
      use_entities: []
  - inform
  - affirm
  - deny
  - stop
  - thankyou
  - greet
  - bot_challenge

entities:
  - destination
  - origin
  - depart_date
  - return_date
  - budget

slots:
  destination:
    type: unfeaturized
    auto_fill: false
  origin:
    type: unfeaturized
    auto_fill: false
  depart_date:
    type: unfeaturized
    auto_fill: false
  return_date:
    type: unfeaturized
    auto_fill: false
  budget:
    type: unfeaturized
    auto_fill: false
  requested_slot:
    type: unfeaturized

responses:
  utter_ask_destination:
    - text: "where do you want to go?"
  utter_ask_origin:
    - text: "where are you leaving from?"
  utter_ask_depart_date:
    - text: "when did you want to leave?"
  utter_ask_return_date:
    - text: "when did you want to be back?"
  utter_ask_budget:
    - text: "what is your budget"
  utter_slots_values:
    - text: "I am going to look for flight offers with the info you gave me:\n
             - destination: {destination}\n
             - origin: {origin}\n
             - departing: {depart_date}\n
             - returning: {return_date}\n
             - budget: {budget}"
  utter_noworries:
    - text: "you are welcome :)"
  utter_chitchat:
    - text: "chitchat"
  utter_ask_continue:
    - text: "do you want to continue?"
  utter_default:
    - text: "sorry, I didn't understand you, please try something else"
  utter_greet:
    - text: "Hello! I am flight search assistant! How can I help?"
  utter_iamabot:
    - text: "I am a bot, powered by Rasa."
  utter_submit:  
  - text: "All done!"

forms:
  - flight_form

session_config:
  session_expiration_time: 60  # value in minutes
  carry_over_slots_to_new_session: true
'''
%store domain_yml > rasa_bot/domain.yml

Writing 'domain_yml' (str) to file 'rasa_bot/domain.yml'.


## Endpoints

In [116]:
endpoints_yml = '''
action_endpoint:
    url: http://localhost:5055/webhook
'''

%store endpoints_yml > rasa_bot/endpoints.yml

Writing 'endpoints_yml' (str) to file 'rasa_bot/endpoints.yml'.


## Custom Action

In [46]:
from rasa.core.actions import Action
from rasa.core.events import SlotSet
import requests

class ApiAction(Action):
    def name(self):
        return 'action_flight_offer'
    
    def submit(self, dispatcher,tracker,domain):
        destination = tracker.get_slot('destination')
        origin = tracker.get_slot('origin')
        depart_date = tracker.get_slot('depart_date')
        return_date = tracker.get_slot('return_date')
        budget = tracker.get_slot('budget')
        
        dispatcher.utter_message(f'Testing:{destination},{origin},{depart_date},{return_date},{budget}')

## Training dialogue model

In [42]:
from rasa.core.policies.fallback import FallbackPolicy
from rasa.core.policies.keras_policy import KerasPolicy
from rasa.core.policies.memoization import MemoizationPolicy
from rasa.core.agent import Agent
import asyncio

async def train_dialogue(domain_file='domain.yml',
                   model_path='models/dialogue',
                   training_data_file='stories.md'):
    
    fallback = FallbackPolicy(fallback_action_name="utter_unclear",
                          core_threshold=0.2,
                          nlu_threshold=0.1)
    
    agent = Agent(domain_file, policies=[MemoizationPolicy(),
                                         KerasPolicy(epochs=200),
                                         fallback])

#     loop = asyncio.get_event_loop()
#     data = loop.run_until_complete(agent.load_data(training_data_file))
    asyncio.run(agent.load_data(training_data_file))

    agent.train(data)

    agent.persist(model_path)
    return agent

In [43]:
from rasa.core.interpreter import RasaNLUInterpreter
def run_travel_bot(serve_forever=True):
    interpreter = RasaNLUInterpreter('./models/nlu/current')
    agent = Agent.load('./models/dialogue', interpreter=interpreter)
    #rasa.core.run.serve_application(agent), channel='cmdline')

    return agent

In [58]:
interpreter = RasaNLUInterpreter('./models/nlu/current')
agent = Agent.load('./models/dialogue', interpreter=interpreter)

INFO:rasa.nlu.components:Added 'SpacyNLP' to component cache. Key 'SpacyNLP-en'.
INFO:rasa.core.policies.ensemble:MappingPolicy not included in policy ensemble. Default intents 'restart and back will not trigger actions 'action_restart' and 'action_back'.


In [72]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])

Your bot is ready to talk! Type your messages here or send 'stop'
hi


TypeError: 'coroutine' object is not iterable

## Testing

In [63]:
import nest_asyncio

nest_asyncio.apply()
print('Event loop ready.')

Event loop ready.


In [64]:
from rasa.cli.scaffold import create_initial_project
import os

project = "test-project"
create_initial_project(project)

# move into project directory and show files
os.chdir(project)
print(os.listdir("."))

['tests', 'actions.py', '__init__.py', '__pycache__', 'endpoints.yml', 'credentials.yml', 'config.yml', 'domain.yml', 'data']


In [65]:
config = "config.yml"
training_files = "stories.md"
domain = "domain.yml"
output = "models/"
print(config, training_files, domain, output)

config.yml data/ domain.yml models/


In [66]:
import rasa

model_path = rasa.train(domain, config, [training_files], output)
print(model_path)

Processed Story Blocks: 100%|██████████| 5/5 [00:00<00:00, 74.41it/s, # trackers=20]

Training Core model...



Processed trackers: 100%|██████████| 5/5 [00:00<00:00, 35.80it/s, # actions=16]
Processed actions: 16it [00:00, 180.57it/s, # examples=16]
Epochs: 100%|██████████| 100/100 [00:13<00:00,  8.19it/s, t_loss=0.081, loss=0.015, acc=1.000]
INFO:rasa.utils.tensorflow.models:Finished training.
INFO:rasa.core.agent:Persisted model to '/var/folders/3t/43w7dzn908qdr0k0s0n6w4f80000gn/T/tmpqj6r2e8q/core'
INFO:rasa.nlu.training_data.training_data:Training data stats: 
	- intent examples: 43 (7 distinct intents)
	- Found intents: 'greet', 'affirm', 'goodbye', 'mood_unhappy', 'bot_challenge', 'mood_great', 'deny'
	- Number of response examples: 0 (0 distinct response)
	- entity examples: 0 (0 distinct entities)
	- found entities: 

INFO:rasa.nlu.model:Starting to train component WhitespaceTokenizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component RegexFeaturizer
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train compo

Core model training completed.
Training NLU model...


INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component DIETClassifier
Epochs: 100%|██████████| 100/100 [00:15<00:00,  5.87it/s, t_loss=1.916, i_loss=0.546, e_loss=0.000, i_acc=1.000, e_f1=0.000]  
INFO:rasa.utils.tensorflow.models:Finished training.
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component EntitySynonymMapper
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Starting to train component ResponseSelector
INFO:rasa.nlu.selectors.response_selector:Retrieval intent parameter was left to its default value. This response selector will be trained on training examples combining all retrieval intents.
INFO:rasa.nlu.model:Finished training component.
INFO:rasa.nlu.model:Successfully saved model into '/var/folders/3t/43w7dzn908qdr0k0s0n6w4f80000gn/T/tmpqj6r2e8q/nlu'


NLU model training completed.
Your Rasa model is trained and saved at '/Users/gthorani/unit_6/travel_chatbot/test-project/models/20200418-234242.tar.gz'.
models/20200418-234242.tar.gz


In [68]:
import rasa.data as data
stories_directory, nlu_data_directory = data.get_core_nlu_directories(training_files)
print(stories_directory, nlu_data_directory)

/var/folders/3t/43w7dzn908qdr0k0s0n6w4f80000gn/T/tmprlorekge /var/folders/3t/43w7dzn908qdr0k0s0n6w4f80000gn/T/tmpqd3e_wpl


In [69]:
rasa.test(model_path, stories_directory, nlu_data_directory)
print("Done testing.")

Processed Story Blocks: 100%|██████████| 5/5 [00:00<00:00, 366.05it/s, # trackers=1]
INFO:rasa.core.test:Evaluating 5 stories
Progress:
100%|██████████| 5/5 [00:00<00:00, 52.23it/s]
INFO:rasa.core.test:Finished collecting predictions.
INFO:rasa.core.test:Evaluation Results on CONVERSATION level:
INFO:rasa.core.test:	Correct:          5 / 5
INFO:rasa.core.test:	F1-Score:         1.000
INFO:rasa.core.test:	Precision:        1.000
INFO:rasa.core.test:	Accuracy:         1.000
INFO:rasa.core.test:	In-data fraction: 1
INFO:rasa.core.test:Evaluation Results on ACTION level:
INFO:rasa.core.test:	Correct:          22 / 22
INFO:rasa.core.test:	F1-Score:         1.000
INFO:rasa.core.test:	Precision:        1.000
INFO:rasa.core.test:	Accuracy:         1.000
INFO:rasa.core.test:	In-data fraction: 1
INFO:rasa.core.test:	Classification report: 
                     precision    recall  f1-score   support

      utter_iamabot       1.00      1.00      1.00         1
     utter_cheer_up       1.00     

Done testing.


In [70]:
if os.path.isfile("errors.json"):
    print("NLU Errors:")
    print(open("errors.json").read())
else:
    print("No NLU errors.")

if os.path.isdir("results"):
      print("\n")
      print("Core Errors:")
      print(open("results/failed_stories.md").read())

No NLU errors.


Core Errors:
<!-- All stories passed -->
